In [3]:
from transformers import pipeline
import pandas as pd

emotion = pipeline('text-classification', 
                    model='arpanghoshal/EmoRoBERTa')

## reference: https://huggingface.co/arpanghoshal/EmoRoBERTa/tree/main
## MIT Licensed 

def get_emotion_score(text):
    return emotion(text)[0]


404 Client Error: Not Found for url: https://huggingface.co/arpanghoshal/EmoRoBERTa/resolve/main/pytorch_model.bin
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [15]:
df20 = pd.read_csv('../data/processed/df_2020_ohe.csv', usecols=['row hash','priorities and concerns','important learning','comments'])
df21 = pd.read_csv('../data/processed/df_2021_ohe.csv', usecols=["Can you please tell us your organisations current key priorities and or concerns",
                                                                 'Can you tell us a short story about your experiences',
                                                                 'Is there anything else you would like to share with us or comment on'])

def dataframe_generator(data, col, year):
    temp = pd.DataFrame()
    for c in col:
        comment_list = data[c].dropna().to_list()
        emotion_list = []
        score_list = []
        for item in comment_list:
            tmp = get_emotion_score(item)
            emotion_list.append(tmp['label'])
            score_list.append(tmp['score'])
        item = [c for comment in comment_list]
        df = pd.DataFrame({"Field":item,
                           "text": comment_list, 
                           "emotions": emotion_list, 
                           "score":score_list})
        temp = pd.concat([temp, df])
    temp.to_csv(f'../data/processed/emotions/{year}_emotions.csv')
    temp = pd.DataFrame(None)

In [16]:
dataframe_generator(df20, ['priorities and concerns','important learning','comments'], 2020)

In [17]:
dataframe_generator(df21, ["Can you please tell us your organisations current key priorities and or concerns",
                           'Can you tell us a short story about your experiences','Is there anything else you would like to share with us or comment on'], 2021)